In [1]:
import numpy as np
import cv2

In [2]:
path = '/mnt/hgfs/Shared/Image/'
pics = ['10.bmp', '11.bmp', '12.bmp']
ref_sizes = [(3, 4), (6, 7), (6, 6)]

In [3]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [4]:
def process(pic, ref_size, criteria):
    img = cv2.imread(path+pic)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray', gray)
    # if cv2.waitKey():
    #     cv2.destroyAllWindows()
    
    objp = np.zeros((ref_size[0]*ref_size[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:ref_size[0],0:ref_size[1]].T.reshape(-1,2)
    
    ret, corners = cv2.findChessboardCorners(gray, ref_size, None)
    # size = (actual_num_column-1, actual_num_row-1)
    
    # refine the cornor locations
    corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

    # img = cv2.drawChessboardCorners(img, ref_size, corners2, ret)
    # cv2.imwrite(pattern + 'res_' + pic, img)
    # cv2.waitKey(0)
    
    objpoints = []
    imgpoints = []
    objpoints.append(objp)
    imgpoints.append(corners)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    cv2.imwrite(path+'res_'+pic,dst)
    
    return ret, mtx, dist, rvecs, tvecs

In [5]:
for i in range(1, len(pics)):
    ret, mtx, dist, rvecs, tvecs = process(pics[i], ref_sizes[i], criteria)
    print(mtx)

[[2.46627017e+03 0.00000000e+00 1.01369221e+03]
 [0.00000000e+00 2.38916557e+03 1.17248422e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[2.92877046e+03 0.00000000e+00 1.13933613e+03]
 [0.00000000e+00 2.91039292e+03 9.80192100e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
